In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-189698
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-189698


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "canina-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('This compute target already exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                          max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    print('A new compute target has been created.')
    
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

A new compute target has been created.
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.05, 10),
    "max_iter": choice(100, 300, 500, 700, 1000)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=3)

if "training" not in os.listdir():
    os.mkdir("./training")

#### Copy train.py script into "./training" to use it in ScriptRunConfig object.
import shutil

shutil.copy('train.py', "./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./training',
                     script='train.py',
                     compute_target=compute_target,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=24,
                                    max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

In [5]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
# We need to wait before retrieving the best model
hyperdrive_run.wait_for_completion(show_output=True)

best_run_hyperdrive = hyperdrive_run.get_best_run_by_primary_metric()
# Print best metrics 
best_metrics = best_run_hyperdrive.get_metrics()
print('Run Id : ', best_run_hyperdrive.id)
print('Metrics :',best_metrics)

# save the model
best_hyperdrive_model = best_run_hyperdrive.register_model(model_name = 'hyperdrivemodel', model_path = 'outputs/model.joblib')

RunId: HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7
Web View: https://ml.azure.com/runs/HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-189698/workspaces/quick-starts-ws-189698&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7
Web View: https://ml.azure.com/runs/HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-189698/workspaces/quick-starts-ws-189698&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Run Id :  HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7_23
Metrics : {'Regularization Strength:': 6.463739391157785, 'Max iterations:': 300, 'Accuracy': 0.9095599393019727}


In [7]:
# This print is to see the best model
print(best_hyperdrive_model.name, best_hyperdrive_model.id, best_hyperdrive_model.version, sep='\t')

hyperdrivemodel	hyperdrivemodel:1	1


In [9]:
best_run_hyperdrive.get_file_names()

['logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/rslex.log',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'user_logs/std_log.txt']

In [13]:
best_run_hyperdrive.download_file(best_run_hyperdrive.get_file_names()[3],
                                 output_file_path = './outputs/')

In [14]:
joblib.load('./outputs/model.joblib')

LogisticRegression(C=6.463739391157785, max_iter=300)

In [15]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
path_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path_data, validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False, encoding='utf8')
    

In [16]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [24]:
from azureml.core.dataset import Dataset
x_ = Dataset.from_pandas_dataframe(x)
y_ = Dataset.from_pandas_dataframe(y)

"Dataset.from_pandas_dataframe" is deprecated after version 1.0.69. Please use "Dataset.Tabular.register_pandas_dataframe" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


In [30]:
ds_ = x.join(y)

In [31]:
ds_ = Dataset.from_pandas_dataframe(ds_)

In [38]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_settings = {
    "experiment_timeout_minutes" : 30,
    "enable_early_stopping" : True,
    "primary_metric" : 'r2_score',
    "featurization" : 'auto',
    "verbosity" : logging.INFO,
    "n_cross_validations" : 5    
}

# Split data into train and test sets using parameter test_size.

automl_config = AutoMLConfig(
    task='regression',
    debug_log='automl_errors.log',
    compute_target=compute_target,
    training_data=ds_,
    label_column_name='y',   
    **automl_settings)

In [39]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on canina-cluster with default configuration
Running on remote compute: canina-cluster


InvalidValueException: InvalidValueException:
	Message: Assertion Failed. Argument dataset_json is null. Target: dataset_json
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "SystemError",
        "message": "Encountered an internal AutoML error. Error Message/Code: InvalidValueException. Additional Info: InvalidValueException:\n\tMessage: Assertion Failed. Argument dataset_json is null. Target: dataset_json\n\tInnerException: None\n\tErrorResponse \n{\n    \"error\": {\n        \"message\": \"Assertion Failed. Argument dataset_json is null. Target: dataset_json\",\n        \"target\": \"dataset_json\"\n    }\n}",
        "details_uri": "https://aka.ms/automltroubleshoot",
        "target": "dataset_json",
        "inner_error": {
            "code": "ClientError",
            "inner_error": {
                "code": "AutoMLInternal"
            }
        }
    }
}

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

automl_run.wait_for_completion(show_output=False)

best_run_automl, best_automl_model = automl_run.get_output()

automl_model_name = best_run_automl.properties["model_name"]
registered_model = automl_run.register_model(model_name=automl_model_name,
                                                 description = "Automl model",
                                                 tags=None)


In [ ]:
test_run_automl = next(best_run_automl.get_children(type='automl.model_test'))
test_run_automl.wait_for_completion(show_output=False, wait_post_processing=True)

In [ ]:
# Test the performance of the automl model.
test_run_metrics = test_run_automl.get_metrics()
for m, val in test_run_metrics.items():
    print(f"{m}: {val}")

In [ ]:
# Remove the compute cluster following training.
compute_target.delete()